# Automate manual validation 

In [1]:
def data_preparation_1():
   
    path_file = path + 'Beer_Wine_Spirits_ML and RF.xlsx'
    df_org = pd.read_excel(path_file)
    df_org = df_org.rename_axis('row_num').reset_index()
    return df_org

In [2]:
def data_preparation_2(df_org):
 
    df = df_org.copy()
    keep_col = ['row_num', 'PMY_DPT_DSC', 'REC_DPT_DSC', 'DPT_DSC', 'COM_DSC', 'SUBCOM_DSC','VND_ECOM_DSC', 'ML L1 Predictions',
           'ML L2 Predictions', 'RF Level 1', 'RF Level 2']
    df = df[keep_col]
    l_val_ml_rf = [ 'VML1', 'VML2','VRF1', 'VRF2']
    l_mth_ml_rf_ind = [ 'VML1_MTH_IND', 'VML2_MTH_IND', 'VRF1_MTH_IND', 'VRF2_MTH_IND']
    l_new_var = l_val_ml_rf + l_mth_ml_rf_ind
    
    for fea in l_val_ml_rf:
        df[fea] = '???' 
        
    for fea in l_mth_ml_rf_ind:   
        df[fea] = False
    df['MANU_VAL_REQ'] = 'NO'
    df['MANU_VAL_DONE'] = 'No Need'   
    return df, l_new_var

In [3]:
def set_indicator(df): 
    df.loc[df['ML L1 Predictions'] == df['VML1'] , 'VML1_MTH_IND'] = 'True'
    df.loc[df['ML L2 Predictions'] == df['VML2'] , 'VML2_MTH_IND'] = 'True'
    df.loc[df['RF Level 1'] == df['VRF1'] , 'VRF1_MTH_IND'] = 'True'
    df.loc[df['RF Level 2'] == df['VRF2'] , 'VRF2_MTH_IND'] = 'True'
    df.loc[(df['VML1_MTH_IND'] == False) |(df['VML2_MTH_IND'] == False) |(df['VRF1_MTH_IND'] == False) |(df['VRF2_MTH_IND'] == False)  , 'MANU_VAL_REQ'] = 'YES'    
    df.loc[df['MANU_VAL_REQ'] == 'YES' , 'MANU_VAL_DONE'] = 'Yes'
    return df

In [4]:
def merge_file(df):
    l_val_df= [df_beer, df_spirits, df_wine, df_hbc, df_gm]
    df_val = pd.concat(l_val_df, ignore_index=True, sort=False)
    keep_col = ['row_num','VML1', 'VML2', 'VRF1', 'VRF2', 'VML1_MTH_IND', 'VML2_MTH_IND', 'VRF1_MTH_IND', 'VRF2_MTH_IND',
               'MANU_VAL_REQ', 'MANU_VAL_DONE']
      
    df_val = df_val[keep_col]
    
    df_merged = pd.merge(df,
                    df_val,
                    on='row_num', how='left')
    df_merged.to_excel(path + "Beer_Wine_Spirits_ML_RF_val.xlsx",index=None)
    return df_merged   

In [5]:
def process_beer(df):
    df_beer = df[(df["PMY_DPT_DSC"] == 'GROCERY') & (df["REC_DPT_DSC"] == 'LIQUOR') & (df["DPT_DSC"] == 'BEER')]
    df_beer.loc[df["DPT_DSC"] == 'BEER','VML1'] = 'Beer / Wine / Spirits'
    df_beer.loc[df["DPT_DSC"] == 'BEER','VRF1'] = 'Beer / Wine / Spirits'
    l2_key_word  = ['BEER', 'BEERS', 'PREMIUM', 'NON-ALCOHOLIC', 'SINGLES','SINGLES']                                                         
    for key_word in l2_key_word:  
        df_beer.loc[(df_beer['COM_DSC'].str.contains(key_word)) | (df_beer['SUBCOM_DSC'].str.contains(key_word)), ['VML2', 'VRF2']] = 'Beer'
    
    df_beer.loc[df_beer['SUBCOM_DSC'].str.contains('CIDER'), ['VML2', 'VRF2']] = 'Ciders & Perrys'   
    df_beer.loc[(df_beer['COM_DSC'].str.contains('BEVERAGES')) | (df_beer['SUBCOM_DSC'].str.contains('BEVERAGES')), ['VML2', 'VRF2']] = 'Flavored Alcoholic Beverages'
    
    df_beer.loc[df_beer['ML L1 Predictions'] == df_beer['VML1'] , 'VML1_MTH_IND'] = 'True'
    df_beer.loc[df_beer['ML L2 Predictions'] == df_beer['VML2'] , 'VML2_MTH_IND'] = 'True'
    df_beer.loc[df_beer['RF Level 1'] == df_beer['VRF1'] , 'VRF1_MTH_IND'] = 'True'
    df_beer.loc[df_beer['RF Level 2'] == df_beer['VRF2'] , 'VRF2_MTH_IND'] = 'True'
    
    df_beer.loc[(df_beer['VML1_MTH_IND'] == False) |(df_beer['VML2_MTH_IND'] == False) |(df_beer['VRF1_MTH_IND'] == False) |(df_beer['VRF2_MTH_IND'] == False)  , 'MANU_VAL_REQ'] = 'YES'
    df_beer['MANU_VAL_REQ'].value_counts()
    df_beer.loc[df_beer['MANU_VAL_REQ'] == 'YES' , 'MANU_VAL_DONE'] = 'No'

    return df_beer

In [6]:
def process_spirits(df):
    df_spc = df[(df["PMY_DPT_DSC"] == 'GROCERY') & (df["REC_DPT_DSC"] == 'LIQUOR') & (df["DPT_DSC"] == 'SPIRITS')]
    df_spc.loc[df["DPT_DSC"] == 'SPIRITS','VML1'] = 'Beer / Wine / Spirits'
    df_spc.loc[df["DPT_DSC"] == 'SPIRITS','VRF1'] = 'Beer / Wine / Spirits'
    
    l2_key_word  =['TEQUILA', 'VODKA',  'WHISKEY', 'RUM','SCOTCH', 'GIN', 'CORDIALS', 'BRANDY']

    for key_word in l2_key_word:
        df_spc.loc[(df_spc['COM_DSC'].str.contains(key_word)) | (df_spc['SUBCOM_DSC'].str.contains(key_word)), ['VML2', 'VRF2']] = 'Spirits' 
    
    df_spc.loc[df_spc['COM_DSC'].str.contains('RTD COCKTAILS'), ['VML2', 'VRF2']] = 'Flavored Alcoholic Beveragess'
    df_spc.loc[df_spc['SUBCOM_DSC'].str.contains('SPIRIT NON ALCOHOL'), ['VML2', 'VRF2']] = 'Flavored Alcoholic Beveragess'

    df_spc.loc[df_spc['ML L1 Predictions'] == df_spc['VML1'] , 'VML1_MTH_IND'] = 'True'
    df_spc.loc[df_spc['ML L2 Predictions'] == df_spc['VML2'] , 'VML2_MTH_IND'] = 'True'
    df_spc.loc[df_spc['RF Level 1'] == df_spc['VRF1'] , 'VRF1_MTH_IND'] = 'True'
    df_spc.loc[df_spc['RF Level 2'] == df_spc['VRF2'] , 'VRF2_MTH_IND'] = 'True'
    
    df_spc.loc[(df_spc['VML1_MTH_IND'] == False) |(df_spc['VML2_MTH_IND'] == False) |(df_spc['VRF1_MTH_IND'] == False) |(df_spc['VRF2_MTH_IND'] == False)  , 'MANU_VAL_REQ'] = 'YES'
    df_spc['MANU_VAL_REQ'].value_counts()
    df_spc.loc[df_spc['MANU_VAL_REQ'] == 'YES' , 'MANU_VAL_DONE'] = 'No'

    return df_spc

In [7]:
def process_wine(df_spc):
    df_spc = df[(df["PMY_DPT_DSC"] == 'GROCERY') & (df["REC_DPT_DSC"] == 'LIQUOR') & (df["DPT_DSC"] == 'WINE')]
    df_spc.loc[df["DPT_DSC"] == 'WINE','VML1'] = 'Beer / Wine / Spirits'
    df_spc.loc[df["DPT_DSC"] == 'WINE','VRF1'] = 'Beer / Wine / Spirits'
    df_spc.loc[df["DPT_DSC"] == 'WINE','VML2'] = 'Wine'
    df_spc.loc[df["DPT_DSC"] == 'WINE','VRF2'] = 'Wine'
    df_spc.loc[(df_spc['VND_ECOM_DSC'].str.lower().str.contains('margarita|rtd|rtd|cocktail|ccktl')), ['VML2', 'VRF2']] = 'Flavored Alcoholic Beverages'
        
    df_spc = set_indicator(df_spc)  
    df_spc.VML2.value_counts() 
    return df_spc

In [8]:
def process_hbc(df_spc):
    df_spc = df[(df["PMY_DPT_DSC"] == 'DRUG/GM') & (df["REC_DPT_DSC"] == 'HBC')]
    
    df_spc.loc[df_spc["DPT_DSC"] == 'HEALTH',['VML1', 'VRF1']] = 'Health & Beauty'
    df_spc.loc[(df_spc["DPT_DSC"] == 'HEALTH') & (df_spc['COM_DSC']=='SPORTS NUTRITION') ,['VML1', 'VRF1']] = 'Food / Beverages'
    df_spc.loc[(df_spc["DPT_DSC"] == 'HEALTH') & (df_spc['COM_DSC']=='SPORTS NUTRITION') ,['VML2', 'VRF2']] = 'Beverages'
    df_spc.loc[(df_spc["DPT_DSC"] == 'HEALTH') & (df_spc['COM_DSC']=='SPORTS NUTRITION') & (df_spc['SUBCOM_DSC'].str.contains('BAR')), ['VML2', 'VRF2']] = 'Grocery'
    df_spc.loc[(df_spc["DPT_DSC"] == 'HEALTH') & (df_spc['SUBCOM_DSC'].str.contains('GROOMING')) ,['VML2', 'VRF2']] = 'Personal Care'
    df_spc.loc[(df_spc["DPT_DSC"] == 'HEALTH') & (df_spc['COM_DSC'].str.contains('FIRST AID PRODUCTS')) ,['VML2', 'VRF2']] = 'Health Treatments'                                          
    df_spc.loc[(df_spc["DPT_DSC"] == 'HEALTH') & (df_spc['COM_DSC']=='VITAMIN PRODUCTS') ,['VML2', 'VRF2']] = 'Vitamins & Dietary Supplements'                                          
    df_spc.loc[(df_spc["DPT_DSC"] == 'HEALTH') & (df_spc['VND_ECOM_DSC'].str.contains('Elix')) ,['VML2', 'VRF2']] = 'Health Treatments'
       
    df_spc.loc[df["DPT_DSC"] == 'BEAUTY',['VML1', 'VRF1']] = 'Health & Beauty'
    df_spc.loc[df_spc['COM_DSC'].str.contains('FRAGRANCES|COLOR COSMETICS|NAILCARE/LASH/COTTON'), ['VML2', 'VRF2']] = 'Cosmetics & Fragrances'
    df_spc.loc[df_spc['COM_DSC'].str.contains('SPECIALTY BATH'), ['VML2', 'VRF2']] = 'Personal Care' 
    
    df_spc.loc[df["DPT_DSC"] == 'CANDY',['VML1', 'VRF1']] = 'Food / Beverages'
    df_spc.loc[(df_spc['COM_DSC'].str.contains('CANDY'))  | (df_spc['SUBCOM_DSC'].str.contains('CANDY')), ['VML2', 'VRF2']] = 'Candy'
       
    df_spc.loc[df["DPT_DSC"] == 'CARDS',['VML1', 'VRF1']] = 'Office Supplies'
    df_spc.loc[df_spc['SUBCOM_DSC'].str.contains('CARDS SEASONAL'), ['VML2', 'VRF2']] = 'Greeting Cards & Gift Packaging'
    
    df_spc.loc[df["DPT_DSC"] == 'PERSONAL CARE',['VML1', 'VRF1']] = 'Health & Beauty'
    df_spc.loc[df["DPT_DSC"] == 'PERSONAL CARE',['VML2', 'VRF2']] = 'Personal Care'
    df_spc.loc[df_spc['SUBCOM_DSC'].str.contains('CONTRACEPTIVES'), ['VML2', 'VRF2']] = 'Healthcare'   
    
    df_spc.loc[df["DPT_DSC"] == 'TOBACCO',['VML1', 'VRF1']] = 'Tobacco Products'
    df_spc.loc[df_spc['SUBCOM_DSC'].str.contains('TOBACCO|CIGARS|CIGARETTE'), ['VML2', 'VRF2']] = 'Cigarettes'
    
    df_spc = set_indicator(df_spc)  
    return df_spc

In [9]:
def process_gm(df_spc):
    df_spc = df[(df["PMY_DPT_DSC"] == 'DRUG/GM') & (df["REC_DPT_DSC"] == 'GM')]
    df_spc =  process_gm_1(df_spc)
    df_spc =  process_gm_2(df_spc)
    df_spc =  process_gm_3(df_spc)
    df_spc =  process_gm_4(df_spc)
    df_spc =  process_gm_5(df_spc)
    df_spc =  process_gm_6(df_spc)
    df_spc = set_indicator(df_spc)  
    return df_spc    

In [10]:
def process_gm_1(df_spc):
   
    df_spc.loc[(df_spc["DPT_DSC"] == 'SEASONAL') & (df_spc['COM_DSC']=='CHRISTMAS') & (df_spc['SUBCOM_DSC']=='WRAP/ACCESSORIES') ,['VML1', 'VRF1']] = 'Office Supplies'
    df_spc.loc[(df_spc["DPT_DSC"] == 'SEASONAL') & (df_spc['COM_DSC']=='CHRISTMAS') & (df_spc['SUBCOM_DSC']=='WRAP/ACCESSORIES') ,['VML2', 'VRF2']] = 'Greeting Cards & Gift Packaging'

    df_spc.loc[(df_spc["DPT_DSC"] == 'SEASONAL') & (df_spc['COM_DSC']=='CHRISTMAS') & (df_spc['SUBCOM_DSC']=='NOVELTY AND DECOR') ,['VML1', 'VRF1']] = 'Home & Venue Decoration'
    df_spc.loc[(df_spc["DPT_DSC"] == 'SEASONAL') & (df_spc['COM_DSC']=='CHRISTMAS') & (df_spc['SUBCOM_DSC']=='NOVELTY AND DECOR') ,['VML2', 'VRF2']] = 'Holiday Decorations'
    
    df_spc.loc[(df_spc["DPT_DSC"] == 'SEASONAL') & ((df_spc['COM_DSC'].str.contains('SOUVENIRS|HALLOWEEN')) | (df_spc['SUBCOM_DSC'].str.contains('SOUVENIRS'))) ,['VML1', 'VRF1']] = 'Home & Venue Decoration'
    df_spc.loc[(df_spc["DPT_DSC"] == 'SEASONAL') & ((df_spc['COM_DSC'].str.contains('SOUVENIRS|HALLOWEEN')) | (df_spc['SUBCOM_DSC'].str.contains('SOUVENIRS'))) ,['VML2', 'VRF2']] = 'Special Occasion & Party Supplies'
    
    df_spc.loc[(df_spc["DPT_DSC"] == 'ACCESSORIES') & (df_spc['COM_DSC'].str.contains('BAG')) ,['VML1', 'VRF1']] = 'Apparel'
    df_spc.loc[(df_spc["DPT_DSC"] == 'ACCESSORIES') & (df_spc['COM_DSC'].str.contains('BAG')) ,['VML2', 'VRF2']] = 'Personal Accessories'

    df_spc.loc[(df_spc["DPT_DSC"] == 'DOMESTICS') & ((df_spc['COM_DSC']=='BATH') | (df_spc['SUBCOM_DSC'].str.contains('BATH'))), ['VML1','VRF1']] = 'Kitchen & Bathroom' 
    df_spc.loc[(df_spc["DPT_DSC"] == 'DOMESTICS') & ((df_spc['COM_DSC']=='BATH') | (df_spc['SUBCOM_DSC'].str.contains('BATH'))), ['VML2','VRF2']] = 'Bathroom'
    
    df_spc.loc[(df_spc["DPT_DSC"] == 'FURN/HOME DEC') & (df_spc['COM_DSC']=='HOME ACCESSORIES') & (df_spc['SUBCOM_DSC']=='DECORATIVE ACCENTS') , ['VML1', 'VRF1']] = 'Home & Venue Decoration'   
    df_spc.loc[(df_spc["DPT_DSC"] == 'FURN/HOME DEC') & (df_spc['COM_DSC']=='HOME ACCESSORIES') & (df_spc['SUBCOM_DSC']=='DECORATIVE ACCENTS') , ['VML2', 'VRF2']] = 'Decorative Accessories'       
    df_spc.loc[(df_spc["DPT_DSC"] == 'FURN/HOME DEC') & (df_spc['COM_DSC']=='CANDLES AND GIFTWARE'), ['VML1', 'VRF1']] = 'Home & Venue Decoration'   
    df_spc.loc[(df_spc["DPT_DSC"] == 'FURN/HOME DEC') & (df_spc['COM_DSC']=='CANDLES AND GIFTWARE'), ['VML2', 'VRF2']] = 'Candles & Accessories'     
    df_spc.loc[(df_spc["DPT_DSC"] == 'FURN/HOME DEC') & (df_spc['COM_DSC']=='WALL DECOR AND FRAMES') & (df_spc['SUBCOM_DSC']=='CLOCKS'), ['VML1', 'VRF1']] = 'Home & Venue Decoration'         
    df_spc.loc[(df_spc["DPT_DSC"] == 'FURN/HOME DEC') & (df_spc['COM_DSC']=='WALL DECOR AND FRAMES') & (df_spc['SUBCOM_DSC']=='CLOCKS'), ['VML2', 'VRF2']] = 'Clocks & Thermometers'   

    df_spc.loc[(df_spc["DPT_DSC"] == 'GARDEN-OUTDOOR'),['VML1', 'VRF1']] = 'Gardening & Outdoors'    
    df_spc.loc[(df_spc["DPT_DSC"] == 'GARDEN-OUTDOOR') & (df_spc['COM_DSC'] == 'PLANTERS/ACCESSORIES') & (df_spc['SUBCOM_DSC']== 'GROWER POTS/ACCESSORIES'),['VML2', 'VRF2']] = 'Gardening'
    df_spc.loc[(df_spc["DPT_DSC"] == 'GARDEN-OUTDOOR') & (df_spc['COM_DSC'] == 'BIRD/SQRL') & (df_spc['SUBCOM_DSC']== 'FEEDERS & ACCESSORIES'),['VML1', 'VRF1']] = 'Livestock & Pet Supplies'    
    df_spc.loc[(df_spc["DPT_DSC"] == 'GARDEN-OUTDOOR') & (df_spc['COM_DSC'] == 'BIRD/SQRL') & (df_spc['SUBCOM_DSC']== 'FEEDERS & ACCESSORIES'),['VML2', 'VRF2']] = 'Pet Supplies'    
    df_spc.loc[(df_spc["DPT_DSC"] == 'GARDEN-OUTDOOR') & (df_spc['COM_DSC'] == 'FIREWOOD/FIRESTARTS') & (df_spc['SUBCOM_DSC']== 'FIREWOOD'),['VML2', 'VRF2']] = 'Outdoor Living' 
    df_spc.loc[(df_spc["DPT_DSC"] == 'GARDEN-OUTDOOR') & (df_spc['COM_DSC'] == 'GARDEN POWER EQUIP') & (df_spc['SUBCOM_DSC']== 'GARDEN POWER TOOLS'),['VML2', 'VRF2']] = 'Outdoor Power Equipment'                                                                                                
    df_spc.loc[(df_spc["DPT_DSC"] == 'GARDEN-OUTDOOR') & (df_spc['COM_DSC'] == 'GARDEN TOOLS/HDW') & (df_spc['SUBCOM_DSC'].str.contains('ENCE/EDGE/FABRIC|GARDEN APPAREL')), ['VML2', 'VRF2']] = 'Outdoor Living'                          
  
    return df_spc

In [11]:
def process_gm_2(df_spc):
    df_spc.loc[(df_spc["DPT_DSC"] == 'GM-MISC') & (df_spc['COM_DSC'] == 'CHARCOAL & LIGHTER FLUID') & (df_spc['SUBCOM_DSC'].str.contains('CHARCOAL|MATCHES')), ['VML1', 'VRF1']] = 'Gardening & Outdoors' 
    df_spc.loc[(df_spc["DPT_DSC"] == 'GM-MISC') & (df_spc['COM_DSC'] == 'CHARCOAL & LIGHTER FLUID') & (df_spc['SUBCOM_DSC'].str.contains('CHARCOAL|MATCHES')), ['VML2', 'VRF2']] = 'Outdoor Living'  
      
    df_spc.loc[(df_spc["DPT_DSC"] == 'HOME IMPRVMNT') & (df_spc['COM_DSC'] == 'HARDWARE') & (df_spc['SUBCOM_DSC'] == 'FASTENERS')  & (df_spc['VND_ECOM_DSC'].str.contains('Hat Rack|Key Holder|Hook')), ['VML1', 'VRF1']] = 'Home & Venue Decoration' 
    df_spc.loc[(df_spc["DPT_DSC"] == 'HOME IMPRVMNT') & (df_spc['COM_DSC'] == 'HARDWARE') & (df_spc['SUBCOM_DSC'] == 'FASTENERS')  & (df_spc['VND_ECOM_DSC'].str.contains('Hat Rack|Key Holder|Hook')), ['VML2', 'VRF2']] = 'Decorative Accessories'                                                                                                 
    # - 3 items HOME IMPRVMNT
    ##   BASIC POWER TOOLS - 3 items                                                                                        
    df_spc.loc[(df_spc["DPT_DSC"] == 'HOME IMPRVMNT') & (df_spc['COM_DSC'] == 'POWER TOOL/ACCSSR') & (df_spc['SUBCOM_DSC'] == 'BASIC POWER TOOLS')  & (df_spc['VND_ECOM_DSC'].str.contains('HDMI Adapter')), ['VML1', 'VRF1']] = 'Electronics' 
    df_spc.loc[(df_spc["DPT_DSC"] == 'HOME IMPRVMNT') & (df_spc['COM_DSC'] == 'POWER TOOL/ACCSSR') & (df_spc['SUBCOM_DSC'] == 'BASIC POWER TOOLS')  & (df_spc['VND_ECOM_DSC'].str.contains('HDMI Adapter')), ['VML2', 'VRF2']] = 'Computer'                                                                                        
    df_spc.loc[(df_spc["DPT_DSC"] == 'HOME IMPRVMNT') & (df_spc['COM_DSC'] == 'POWER TOOL/ACCSSR') & (df_spc['SUBCOM_DSC'] == 'BASIC POWER TOOLS')  & (df_spc['VND_ECOM_DSC'].str.contains('Beer Growle')), ['VML1', 'VRF1']] = 'Beer / Wine / Spirits' 
    df_spc.loc[(df_spc["DPT_DSC"] == 'HOME IMPRVMNT') & (df_spc['COM_DSC'] == 'POWER TOOL/ACCSSR') & (df_spc['SUBCOM_DSC'] == 'BASIC POWER TOOLS')  & (df_spc['VND_ECOM_DSC'].str.contains('Beer Growle')), ['VML2', 'VRF2']] = 'Alcohol Making Supplies' 
    df_spc.loc[(df_spc["DPT_DSC"] == 'HOME IMPRVMNT') & (df_spc['COM_DSC'] == 'POWER TOOL/ACCSSR') & (df_spc['SUBCOM_DSC'] == 'BASIC POWER TOOLS')  & (df_spc['VND_ECOM_DSC'].str.contains('Handrail')), ['VML1', 'VRF1']] = 'Gardening & Outdoors' 
    df_spc.loc[(df_spc["DPT_DSC"] == 'HOME IMPRVMNT') & (df_spc['COM_DSC'] == 'POWER TOOL/ACCSSR') & (df_spc['SUBCOM_DSC'] == 'BASIC POWER TOOLS')  & (df_spc['VND_ECOM_DSC'].str.contains('Handrail')), ['VML2', 'VRF2']] = 'Outdoor Living'                                                                                               
    
    df_spc.loc[(df_spc["DPT_DSC"] == 'HOME IMPRVMNT') & (df_spc['COM_DSC'] == 'HOUSEHOLD ELECTRIC') & (df_spc['SUBCOM_DSC'] == 'DIY ELECTRICAL')  & (df_spc['VND_ECOM_DSC'].str.contains('Light Sconce')), ['VML1', 'VRF1']] = 'Home & Venue Decoration' 
    df_spc.loc[(df_spc["DPT_DSC"] == 'HOME IMPRVMNT') & (df_spc['COM_DSC'] == 'HOUSEHOLD ELECTRIC') & (df_spc['SUBCOM_DSC'] == 'DIY ELECTRICAL')  & (df_spc['VND_ECOM_DSC'].str.contains('Light Sconce')), ['VML2', 'VRF2']] = 'Decorative Accessories'   
    df_spc.loc[(df_spc["DPT_DSC"] == 'HOME IMPRVMNT') & (df_spc['COM_DSC'] == 'HOUSEHOLD ELECTRIC') & (df_spc['SUBCOM_DSC'] == 'DIY ELECTRICAL')  & (df_spc['VND_ECOM_DSC'].str.contains('Gardner Bender')), ['VML1', 'VRF1']] = 'Electrical' 
    df_spc.loc[(df_spc["DPT_DSC"] == 'HOME IMPRVMNT') & (df_spc['COM_DSC'] == 'HOUSEHOLD ELECTRIC') & (df_spc['SUBCOM_DSC'] == 'DIY ELECTRICAL')  & (df_spc['VND_ECOM_DSC'].str.contains('Gardner Bender')), ['VML2', 'VRF2']] = 'Wire, Cable & Cord' 
    df_spc.loc[(df_spc["DPT_DSC"] == 'HOME IMPRVMNT') & (df_spc['COM_DSC'] == 'HEATERS & FANS') & (df_spc['SUBCOM_DSC'] == 'HEATERS'), ['VML1', 'VRF1']] = 'Heating / Ventilation / Air Conditioning' 
    df_spc.loc[(df_spc["DPT_DSC"] == 'HOME IMPRVMNT') & (df_spc['COM_DSC'] == 'HEATERS & FANS') & (df_spc['SUBCOM_DSC'] == 'HEATERS'), ['VML2', 'VRF2']] = 'Heating' 
    df_spc.loc[(df_spc["DPT_DSC"] == 'HOME IMPRVMNT') & (df_spc['COM_DSC'] == 'AUTO CHEMICALS') & (df_spc['SUBCOM_DSC'] == 'WINDSHIELD'), ['VML1', 'VRF1']] = 'Automotive' 
    df_spc.loc[(df_spc["DPT_DSC"] == 'HOME IMPRVMNT') & (df_spc['COM_DSC'] == 'AUTO CHEMICALS') & (df_spc['SUBCOM_DSC'] == 'WINDSHIELD'), ['VML2', 'VRF2']] = 'Fluids, Chemicals, Compounds'  
    df_spc.loc[(df_spc["DPT_DSC"] == 'HOME IMPRVMNT') & (df_spc['COM_DSC'] == 'AUTO ACCESSORIES') & (df_spc['SUBCOM_DSC'] == 'FALL & WINTER SEASONAL'), ['VML1', 'VRF1']] = 'Automotive' 
    df_spc.loc[(df_spc["DPT_DSC"] == 'HOME IMPRVMNT') & (df_spc['COM_DSC'] == 'AUTO ACCESSORIES') & (df_spc['SUBCOM_DSC'] == 'FALL & WINTER SEASONAL'), ['VML2', 'VRF2']] = 'Fluids, Chemicals, Compounds'  
    df_spc.loc[(df_spc["DPT_DSC"] == 'HOME IMPRVMNT') & (df_spc['COM_DSC'] == 'AUTO ACCESSORIES') & (df_spc['SUBCOM_DSC'] == 'FALL & WINTER SEASONAL') & (df_spc['VND_ECOM_DSC'].str.contains('SNWBRM')), ['VML1', 'VRF1']] = 'Automotive' 
    df_spc.loc[(df_spc["DPT_DSC"] == 'HOME IMPRVMNT') & (df_spc['COM_DSC'] == 'AUTO ACCESSORIES') & (df_spc['SUBCOM_DSC'] == 'FALL & WINTER SEASONAL') & (df_spc['VND_ECOM_DSC'].str.contains('SNWBRM')), ['VML2', 'VRF2']] = 'Tools'                                                                                               
    return df_spc    

In [12]:
 def process_gm_3(df_spc): 
     """
       HOUSEWARES
        - APPLIANCES
        - BAKEWARE 
     """   
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'APPLIANCES') & (df_spc['SUBCOM_DSC'].str.contains('GENERAL USE/MISC')) & (df_spc['VND_ECOM_DSC'].str.contains('Wine Brewing|Beer Growler')), ['VML1', 'VRF1']] = 'Beer / Wine / Spirits' 
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'APPLIANCES') & (df_spc['SUBCOM_DSC'].str.contains('GENERAL USE/MISC')) & (df_spc['VND_ECOM_DSC'].str.contains('Wine Brewing|Beer Growler')), ['VML2', 'VRF2']] = 'Alcohol Making Supplies'  
                                                                                              
     # - 3 items HOUSEWARES
     ##   BASIC POWER TOOLS - 3 items                                                                                        
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'BAKEWARE') & (df_spc['SUBCOM_DSC'] == 'SEASONAL')  & (df_spc['VND_ECOM_DSC'].str.contains('NONPAREILS TALL|FISH|DRNK')), ['VML1', 'VRF1']] = 'Food / Beverages' 
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'BAKEWARE') & (df_spc['SUBCOM_DSC'] == 'SEASONAL')  & (df_spc['VND_ECOM_DSC'].str.contains('NONPAREILS TALL|FISH|DRNK')), ['VML2', 'VRF2']] = 'Grocery'                                                                                        
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'BAKEWARE') & (df_spc['SUBCOM_DSC'] == 'PANTRY')  & (df_spc['VND_ECOM_DSC'].str.contains('Condiment Holder')), ['VML1', 'VRF1']] = 'Kitchen & Bathroom' 
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'BAKEWARE') & (df_spc['SUBCOM_DSC'] == 'PANTRY')  & (df_spc['VND_ECOM_DSC'].str.contains('Condiment Holder')), ['VML2', 'VRF2']] = 'Kitchen'
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'BAKEWARE') & (df_spc['SUBCOM_DSC'] == 'SEASONAL')  & (df_spc['VND_ECOM_DSC'].str.contains('CUPCAKE  BTL STAND')), ['VML1', 'VRF1']] = 'Kitchen & Bathroom' 
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'BAKEWARE') & (df_spc['SUBCOM_DSC'] == 'SEASONAL')  & (df_spc['VND_ECOM_DSC'].str.contains('CUPCAKE  BTL STAND')), ['VML2', 'VRF2']] = 'Kitchen'      
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'BAKEWARE') & (df_spc['SUBCOM_DSC'] == 'SEASONAL')  & (df_spc['VND_ECOM_DSC'].str.contains('Swivel Riser')), ['VML1', 'VRF1']] = 'Office Supplies' 
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'BAKEWARE') & (df_spc['SUBCOM_DSC'] == 'SEASONAL')  & (df_spc['VND_ECOM_DSC'].str.contains('Swivel Riser')), ['VML2', 'VRF2']] = 'Office Machinery'
     
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'BEVERAGE') & (df_spc['SUBCOM_DSC'] == 'COFFEE')  & (df_spc['VND_ECOM_DSC'].str.contains('Mug|Glass Carafe')), ['VML1', 'VRF1']] = 'Kitchen & Bathroom' 
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'BEVERAGE') & (df_spc['SUBCOM_DSC'] == 'COFFEE')  & (df_spc['VND_ECOM_DSC'].str.contains('Mug|Glass Carafe')), ['VML2', 'VRF2']] = 'Kitchen'                                                                                               
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'BEVERAGE') & (df_spc['SUBCOM_DSC'] == 'COFFEE')  & (df_spc['VND_ECOM_DSC'].str.contains('Beer Bong by True')), ['VML1', 'VRF1']] = 'Home & Venue Decoration' 
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'BEVERAGE') & (df_spc['SUBCOM_DSC'] == 'COFFEE')  & (df_spc['VND_ECOM_DSC'].str.contains('Beer Bong by True')), ['VML2', 'VRF2']] = 'Special Occasion & Party Supplies'       
     
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'BEVERAGE') & (df_spc['SUBCOM_DSC'] == 'SPECIALTY BEVERAGE')  & (df_spc['VND_ECOM_DSC'].str.contains('WINE GROWLER')), ['VML1', 'VRF1']] = 'Kitchen & Bathroom' 
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'BEVERAGE') & (df_spc['SUBCOM_DSC'] == 'SPECIALTY BEVERAGE')  & (df_spc['VND_ECOM_DSC'].str.contains('WINE GROWLER')), ['VML2', 'VRF2']] = 'Kitchen'
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'BEVERAGE') & (df_spc['SUBCOM_DSC'] == 'THERMAL BEVERAGEWARE'), ['VML1', 'VRF1']] = 'Kitchen & Bathroom' 
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'BEVERAGE') & (df_spc['SUBCOM_DSC'] == 'THERMAL BEVERAGEWARE'), ['VML2', 'VRF2']] = 'Kitchen'     
    
     
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'BEVERAGE') & (df_spc['SUBCOM_DSC'] == 'SPECIALTY BEVERAGE')  & (df_spc['VND_ECOM_DSC'].str.contains('Beer Bong by True')), ['VML1', 'VRF1']] = 'Home & Venue Decoration' 
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'BEVERAGE') & (df_spc['SUBCOM_DSC'] == 'SPECIALTY BEVERAGE')  & (df_spc['VND_ECOM_DSC'].str.contains('Beer Bong by True')), ['VML2', 'VRF2']] = 'Special Occasion & Party Supplies'                     
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'BEVERAGE') & (df_spc['SUBCOM_DSC'] == 'SPECIALTY BEVERAGE')  & (df_spc['VND_ECOM_DSC'].str.contains('Brewing System')), ['VML1', 'VRF1']] = 'Office Supplies' 
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'BEVERAGE') & (df_spc['SUBCOM_DSC'] == 'SPECIALTY BEVERAGE')  & (df_spc['VND_ECOM_DSC'].str.contains('Brewing System')), ['VML2', 'VRF2']] = 'Office Machinery'            
     
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'DINNERWARE/SERVEWARE') & (df_spc['SUBCOM_DSC'] == 'SEASONAL'), ['VML1', 'VRF1']] = 'Kitchen & Bathroom' 
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'DINNERWARE/SERVEWARE') & (df_spc['SUBCOM_DSC'] == 'SEASONAL'), ['VML2', 'VRF2']] = 'Kitchen'  
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'DINNERWARE/SERVEWARE') & (df_spc['SUBCOM_DSC'] == 'SEASONAL')  & (df_spc['VND_ECOM_DSC'].str.contains('HALLOWEEN ASSORTED HALF PALLET')), ['VML1', 'VRF1']] = 'Home & Venue Decoration' 
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'DINNERWARE/SERVEWARE') & (df_spc['SUBCOM_DSC'] == 'SEASONAL')  & (df_spc['VND_ECOM_DSC'].str.contains('HALLOWEEN ASSORTED HALF PALLET')), ['VML2', 'VRF2']] = 'Special Occasion & Party Supplies' 
     
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'DRINKWARE/FLATWARE') & (df_spc['SUBCOM_DSC'] == 'DRINKWARE')  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('glass|cathys Concepts|decanter|growler|tumbler|hug|bag|caddy|stein|craft|opener|carolines treasures|koozie|markers|sleeves|mug|holder|cooler|wine preserver kit|crystal|spiegelau|wine carafe|beer flight by true')), ['VML1', 'VRF1']] = 'Kitchen & Bathroom' 
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'DRINKWARE/FLATWARE') & (df_spc['SUBCOM_DSC'] == 'DRINKWARE')  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('glass|cathys Concepts|decanter|growler|tumbler|hug|bag|caddy|stein|craft|opener|carolines treasures|koozie|markers|sleeves|mug|holder|cooler|wine preserver kit|crystal|spiegelau|wine carafe|beer flight by true')), ['VML2', 'VRF2']] = 'Kitchen'
       
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'DRINKWARE/FLATWARE') & (df_spc['SUBCOM_DSC'] == 'DRINKWARE')  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('shooter dispenser|beer tower')), ['VML1', 'VRF1']] = 'Hospitality Supplies' 
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'DRINKWARE/FLATWARE') & (df_spc['SUBCOM_DSC'] == 'DRINKWARE')  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('shooter dispenser|beer tower')), ['VML2', 'VRF2']] = 'Dispensers'   
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'DRINKWARE/FLATWARE') & (df_spc['SUBCOM_DSC'] == 'DRINKWARE')  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('wine charms')), ['VML1', 'VRF1']] = 'Home & Venue Decoration'
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'DRINKWARE/FLATWARE') & (df_spc['SUBCOM_DSC'] == 'DRINKWARE')  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('wine charms')), ['VML2', 'VRF2']] = 'Special Occasion & Party Supplies'
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'HEATERS & FANS') & (df_spc['SUBCOM_DSC'] == 'HEATERS'), ['VML1', 'VRF1']] = 'Heating / Ventilation / Air Conditioning' 
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'HEATERS & FANS') & (df_spc['SUBCOM_DSC'] == 'HEATERS'), ['VML2', 'VRF2']] = 'Heating' 
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'AUTO CHEMICALS') & (df_spc['SUBCOM_DSC'] == 'WINDSHIELD'), ['VML1', 'VRF1']] = 'Automotive' 
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'AUTO CHEMICALS') & (df_spc['SUBCOM_DSC'] == 'WINDSHIELD'), ['VML2', 'VRF2']] = 'Fluids, Chemicals, Compounds'  
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'AUTO ACCESSORIES') & (df_spc['SUBCOM_DSC'] == 'FALL & WINTER SEASONAL'), ['VML1', 'VRF1']] = 'Automotive' 
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'AUTO ACCESSORIES') & (df_spc['SUBCOM_DSC'] == 'FALL & WINTER SEASONAL'), ['VML2', 'VRF2']] = 'Fluids, Chemicals, Compounds'  
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'AUTO ACCESSORIES') & (df_spc['SUBCOM_DSC'] == 'FALL & WINTER SEASONAL') & (df_spc['VND_ECOM_DSC'].str.contains('SNWBRM')), ['VML1', 'VRF1']] = 'Automotive' 
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'AUTO ACCESSORIES') & (df_spc['SUBCOM_DSC'] == 'FALL & WINTER SEASONAL') & (df_spc['VND_ECOM_DSC'].str.contains('SNWBRM')), ['VML2', 'VRF2']] = 'Tools'
     
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'DRINKWARE/FLATWARE') & (df_spc['SUBCOM_DSC'] == 'DRINKWARE')  
              & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('glass|cathys concepts|decanter|growler|tumbler|hug|bag|caddy|stein|craft|opener|carolines treasures|koozie|markers|sleeves|mug|holder|cooler|wine preserver kit|crystal|spiegelau|wine carafe|beer flight by true')), ['VML1', 'VRF1']] = 'Kitchen & Bathroom' 
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'DRINKWARE/FLATWARE') & (df_spc['SUBCOM_DSC'] == 'DRINKWARE')  
              & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('glass|cathys concepts|decanter|growler|tumbler|hug|bag|caddy|stein|craft|opener|carolines treasures|koozie|markers|sleeves|mug|holder|cooler|wine preserver kit|crystal|spiegelau|wine carafe|beer flight by true')), ['VML2', 'VRF2']] = 'Kitchen'        
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'DRINKWARE/FLATWARE') & (df_spc['SUBCOM_DSC'] == 'DRINKWARE')  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('shooter dispenser|beer tower')), ['VML1', 'VRF1']] = 'Hospitality Supplies' 
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'DRINKWARE/FLATWARE') & (df_spc['SUBCOM_DSC'] == 'DRINKWARE')  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('shooter dispenser|beer tower')), ['VML2', 'VRF2']] = 'Dispensers'   
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'DRINKWARE/FLATWARE') & (df_spc['SUBCOM_DSC'] == 'DRINKWARE')  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('wine charms')), ['VML1', 'VRF1']] = 'Home & Venue Decoration'
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'DRINKWARE/FLATWARE') & (df_spc['SUBCOM_DSC'] == 'DRINKWARE')  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('wine charms')), ['VML2', 'VRF2']] = 'Special Occasion & Party Supplies'
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'DRINKWARE/FLATWARE') & (df_spc['SUBCOM_DSC'] == 'FLATWARE')   & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('Tier Stand')), ['VML1', 'VRF1']] = 'Kitchen & Bathroom'
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'DRINKWARE/FLATWARE') & (df_spc['SUBCOM_DSC'] == 'FLATWARE')   & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('Tier Stand')), ['VML2', 'VRF2']] = 'Kitchen'
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'DRINKWARE/FLATWARE') & (df_spc['SUBCOM_DSC'] == 'SEASONAL')   & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('glass|gls|goblets|wine box')), ['VML1', 'VRF1']] = 'Kitchen & Bathroom'
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'DRINKWARE/FLATWARE') & (df_spc['SUBCOM_DSC'] == 'SEASONAL')   & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('glass|gls|goblets|wine box')), ['VML2', 'VRF2']] = 'Kitchen'
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'DRINKWARE/FLATWARE') & (df_spc['SUBCOM_DSC'] == 'SEASONAL')  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('wine charms')), ['VML1', 'VRF1']] = 'Home & Venue Decoration'
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'DRINKWARE/FLATWARE') & (df_spc['SUBCOM_DSC'] == 'SEASONAL')  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('wine charms')), ['VML2', 'VRF2']] = 'Special Occasion & Party Supplies'  

     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'FOOD STORAGE') & (df_spc['SUBCOM_DSC'] == 'KITCHEN ACCESSORIES')   & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('cabinet|credenza|cupboard hood')), ['VML1', 'VRF1']] = 'Kitchen & Bathroom'
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'FOOD STORAGE') & (df_spc['SUBCOM_DSC'] == 'KITCHEN ACCESSORIES')   & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('cabinet|credenza|cupboard hood')), ['VML2', 'VRF2']] = 'Cabinets'
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'FOOD STORAGE') & (df_spc['SUBCOM_DSC'] == 'KITCHEN ACCESSORIES')   & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('rack|holder|basket|convenience concepts|floating shelves|decanter|etagere')), ['VML1', 'VRF1']] = 'Kitchen & Bathroom'
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'FOOD STORAGE') & (df_spc['SUBCOM_DSC'] == 'KITCHEN ACCESSORIES')   & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('rack|holder|basket|convenience concepts|floating shelves|decanter|etagere')), ['VML2', 'VRF2']] = 'Kitchen' 
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'FOOD STORAGE') & (df_spc['SUBCOM_DSC'] == 'FOOD')   & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('clamshell')), ['VML1', 'VRF1']] = 'Kitchen & Bathroom'
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'FOOD STORAGE') & (df_spc['SUBCOM_DSC'] == 'FOOD')   & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('clamshell')), ['VML2', 'VRF2']] = 'Kitchen'       
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'FOOD STORAGE') & (df_spc['SUBCOM_DSC'] == 'KITCHEN ACCESSORIES')   & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('bookcase|bookshelf ladder')), ['VML1', 'VRF1']] = 'Office Supplies'
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'FOOD STORAGE') & (df_spc['SUBCOM_DSC'] == 'KITCHEN ACCESSORIES')   & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('bookcase|bookshelf ladder')), ['VML2', 'VRF2']] = 'Office Machinery'         
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'FOOD STORAGE') & (df_spc['SUBCOM_DSC'] == 'KITCHEN ACCESSORIES')  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('cart')), ['VML1', 'VRF1']] = 'Home & Venue Decoration'
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'FOOD STORAGE') & (df_spc['SUBCOM_DSC'] == 'KITCHEN ACCESSORIES')  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('cart')), ['VML2', 'VRF2']] = 'Special Occasion & Party Supplies' 
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'FOOD STORAGE') & (df_spc['SUBCOM_DSC'] == 'KITCHEN ACCESSORIES')  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('gun chest')), ['VML1', 'VRF1']] = 'Sports & Outdoor Recreation Equipment'
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'FOOD STORAGE') & (df_spc['SUBCOM_DSC'] == 'KITCHEN ACCESSORIES')  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('gun chest')), ['VML2', 'VRF2']] = 'Hunting & Fishing'            

     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'TEXTILES') & (df_spc['SUBCOM_DSC'].str.contains('SEASONAL|KITCHEN TEXTILES'))   & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('sack|towl|tc|potholder|holder|ecovinyl|apr spruce happy hanukkah')), ['VML1', 'VRF1']] = 'Kitchen & Bathroom'
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'TEXTILES') & (df_spc['SUBCOM_DSC'].str.contains('SEASONAL|KITCHEN TEXTILES'))   & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('sack|towl|tc|potholder|holder|ecovinyl|apr spruce happy hanukkah')), ['VML2', 'VRF2']] = 'Kitchen'        
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'TEXTILES') & (df_spc['SUBCOM_DSC'] == 'SEASONAL')  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('assorted dollar pallet')), ['VML1', 'VRF1']] = 'Office Supplies'
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'TEXTILES') & (df_spc['SUBCOM_DSC'] == 'SEASONAL')  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('assorted dollar pallet')), ['VML2', 'VRF2']] = 'Office Machinery'  
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'TEXTILES') & (df_spc['SUBCOM_DSC'] == 'SEASONAL')  & (df_spc['VND_ECOM_DSC'].str.contains('HALLOWEEN ASST. SINGLE')), ['VML1', 'VRF1']] = 'Home & Venue Decoration'
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'TEXTILES') & (df_spc['SUBCOM_DSC'] == 'SEASONAL')  & (df_spc['VND_ECOM_DSC'].str.contains('HALLOWEEN ASST. SINGLE')), ['VML2', 'VRF2']] = 'Special Occasion & Party Supplies'        
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'TEXTILES') & (df_spc['SUBCOM_DSC'] == 'SEASONAL')  & (df_spc['VND_ECOM_DSC'].str.contains('BLB TO -GO  OPEN STOCK|Montana Check 70 RD')), ['VML1', 'VRF1']] = 'Misc'
     df_spc.loc[(df_spc["DPT_DSC"] == 'HOUSEWARES') & (df_spc['COM_DSC'] == 'TEXTILES') & (df_spc['SUBCOM_DSC'] == 'SEASONAL')  & (df_spc['VND_ECOM_DSC'].str.contains('BLB TO -GO  OPEN STOCK|Montana Check 70 RD')), ['VML2', 'VRF2']] = 'Misc'        
     return df_spc 

In [13]:
 def process_gm_4(df_spc): 
     """
       DPT_DSC: VIDEO/ELEC/COMP

     """ 
     df_spc.loc[(df_spc["DPT_DSC"] == 'VIDEO/ELEC/COMP') & (df_spc['COM_DSC'] == 'AUDIO') & (df_spc['SUBCOM_DSC'].str.contains('APPLE'))  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('earpod|earphones|adapter|lightning')), ['VML1', 'VRF1']] = 'Electronics'
     df_spc.loc[(df_spc["DPT_DSC"] == 'VIDEO/ELEC/COMP') & (df_spc['COM_DSC'] == 'AUDIO') & (df_spc['SUBCOM_DSC'].str.contains('APPLE'))  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('earpod|earphones|adapter|lightning')), ['VML2', 'VRF2']] = 'Audio & Video' 
     df_spc.loc[(df_spc["DPT_DSC"] == 'VIDEO/ELEC/COMP') & (df_spc['COM_DSC'] == 'AUDIO') & (df_spc['SUBCOM_DSC'].str.contains('APPLE'))  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('drive')), ['VML1', 'VRF1']] = 'Elec'
     df_spc.loc[(df_spc["DPT_DSC"] == 'VIDEO/ELEC/COMP') & (df_spc['COM_DSC'] == 'AUDIO') & (df_spc['SUBCOM_DSC'].str.contains('APPLE'))  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('drive')), ['VML2', 'VRF2']] = 'Computers'        
     df_spc.loc[(df_spc["DPT_DSC"] == 'VIDEO/ELEC/COMP') & (df_spc['COM_DSC'] == 'AUDIO') & (df_spc['SUBCOM_DSC'].str.contains('SPEAKERS')), ['VML1', 'VRF1']] = 'Electronics'
     df_spc.loc[(df_spc["DPT_DSC"] == 'VIDEO/ELEC/COMP') & (df_spc['COM_DSC'] == 'AUDIO') & (df_spc['SUBCOM_DSC'].str.contains('SPEAKERS')), ['VML2', 'VRF2']] = 'Audio & Video'    
       
     df_spc.loc[(df_spc["DPT_DSC"] == 'VIDEO/ELEC/COMP') & (df_spc['COM_DSC'] == 'HOME ELCTRNIC ACCSS') & (df_spc['SUBCOM_DSC'] == 'HOME OFFICE ACCESSORIES')  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('mousepad|mouse pad|pad|scanner')), ['VML1', 'VRF1']] = 'Electronics'
     df_spc.loc[(df_spc["DPT_DSC"] == 'VIDEO/ELEC/COMP') & (df_spc['COM_DSC'] == 'HOME ELCTRNIC ACCSS') & (df_spc['SUBCOM_DSC'] == 'HOME OFFICE ACCESSORIES')  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('mousepad|mouse pad|pad|scanner')), ['VML2', 'VRF2']] = 'Computers'         
     df_spc.loc[(df_spc["DPT_DSC"] == 'VIDEO/ELEC/COMP') & (df_spc['COM_DSC'] == 'HOME OFFICE HARDWARE') & (df_spc['SUBCOM_DSC'] == 'CONNECTED HOME')  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('speaker')), ['VML1', 'VRF1']] = 'Electronics'
     df_spc.loc[(df_spc["DPT_DSC"] == 'VIDEO/ELEC/COMP') & (df_spc['COM_DSC'] == 'HOME OFFICE HARDWARE') & (df_spc['SUBCOM_DSC'] == 'CONNECTED HOME')  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('speaker')), ['VML2', 'VRF2']] = 'Audio & Video'  
     df_spc.loc[(df_spc["DPT_DSC"] == 'VIDEO/ELEC/COMP') & (df_spc['COM_DSC'] == 'HOME OFFICE HARDWARE') & (df_spc['SUBCOM_DSC'] == 'CONNECTED HOME')  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('hat holder')), ['VML1', 'VRF1']] = 'Office Supplies'
     df_spc.loc[(df_spc["DPT_DSC"] == 'VIDEO/ELEC/COMP') & (df_spc['COM_DSC'] == 'HOME OFFICE HARDWARE') & (df_spc['SUBCOM_DSC'] == 'CONNECTED HOME')  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('hat holder')), ['VML2', 'VRF2']] = 'Office Machinery' 
     df_spc.loc[(df_spc["DPT_DSC"] == 'VIDEO/ELEC/COMP') & (df_spc['COM_DSC'] == 'HOME OFFICE HARDWARE') & (df_spc['SUBCOM_DSC'] == 'COMPUTER DEVICES')  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('adapt')), ['VML1', 'VRF1']] = 'Electronics'
     df_spc.loc[(df_spc["DPT_DSC"] == 'VIDEO/ELEC/COMP') & (df_spc['COM_DSC'] == 'HOME OFFICE HARDWARE') & (df_spc['SUBCOM_DSC'] == 'COMPUTER DEVICES')  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('adapt')), ['VML2', 'VRF2']] = 'Computers'         
     df_spc.loc[(df_spc["DPT_DSC"] == 'VIDEO/ELEC/COMP') & (df_spc['COM_DSC'] == 'HOME OFFICE HARDWARE') & (df_spc['SUBCOM_DSC'] == 'COMPUTER DEVICES')  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('network cabinet')), ['VML1', 'VRF1']] = 'Electronics'
     df_spc.loc[(df_spc["DPT_DSC"] == 'VIDEO/ELEC/COMP') & (df_spc['COM_DSC'] == 'HOME OFFICE HARDWARE') & (df_spc['SUBCOM_DSC'] == 'COMPUTER DEVICES')  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('network cabinet')), ['VML2', 'VRF2']] = 'Security Systems & Components'  
     df_spc.loc[(df_spc["DPT_DSC"] == 'VIDEO/ELEC/COMP') & (df_spc['COM_DSC'] == 'VIDEO HARDWARE') & (df_spc['SUBCOM_DSC'] == 'BD/DVD/STREAMING DEVICES')  & (df_spc['VND_ECOM_DSC'].str.contains('LAST STAND AT SABER RIV')), ['VML1', 'VRF1']] = 'Books & Videos'
     df_spc.loc[(df_spc["DPT_DSC"] == 'VIDEO/ELEC/COMP') & (df_spc['COM_DSC'] == 'VIDEO HARDWARE') & (df_spc['SUBCOM_DSC'] == 'BD/DVD/STREAMING DEVICES')  & (df_spc['VND_ECOM_DSC'].str.contains('LAST STAND AT SABER RIV')), ['VML2', 'VRF2']] = 'Film & Television'   
        
     df_spc.loc[(df_spc["DPT_DSC"] == 'MUSIC MKT') & (df_spc['COM_DSC'] == 'DVDS'), ['VML1', 'VRF1']] = 'Books & Videos'
     df_spc.loc[(df_spc["DPT_DSC"] == 'MUSIC MKT') & (df_spc['COM_DSC'] == 'DVDS'), ['VML2', 'VRF2']] = 'Film & Television'  
     df_spc.loc[(df_spc["DPT_DSC"] == 'MUSIC MKT') & (df_spc['COM_DSC'] == 'DVDS') & (df_spc['SUBCOM_DSC'] == "DVD'S")  & (df_spc['VND_ECOM_DSC'].str.contains('LEAPFROG')), ['VML1', 'VRF1']] = 'Toys / Games / Hobbies'
     df_spc.loc[(df_spc["DPT_DSC"] == 'MUSIC MKT') & (df_spc['COM_DSC'] == 'DVDS') & (df_spc['SUBCOM_DSC'] == "DVD'S")  & (df_spc['VND_ECOM_DSC'].str.contains('LEAPFROG')), ['VML2', 'VRF2']] = 'Games & Puzzles'        
     df_spc.loc[(df_spc["DPT_DSC"] == 'MUSIC MKT') & (df_spc['COM_DSC'] == 'VIDEO ACCESSORIES') & (df_spc['SUBCOM_DSC'] == "CONSOLE ACCESSORIES")  & (df_spc['VND_ECOM_DSC'].str.contains('DecalGirl PS3S-ALE PS3 Slim Skin - Beer Bubbles')), ['VML1', 'VRF1']] = 'Electronics'
     df_spc.loc[(df_spc["DPT_DSC"] == 'MUSIC MKT') & (df_spc['COM_DSC'] == 'VIDEO ACCESSORIES') & (df_spc['SUBCOM_DSC'] == "CONSOLE ACCESSORIES")  & (df_spc['VND_ECOM_DSC'].str.contains('DecalGirl PS3S-ALE PS3 Slim Skin - Beer Bubbles')), ['VML2', 'VRF2']] = 'Computers'        
        
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'CRAFTS/NOTIONS/YARN') & (df_spc['SUBCOM_DSC'] == "NOTIONS & YARN")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains("knit|crochet|needle|stitch kit|ptrn|batting santa sock|sewing|sugar'n cream|c&c|zipper")), ['VML1', 'VRF1']] = 'Arts & Crafts'
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'CRAFTS/NOTIONS/YARN') & (df_spc['SUBCOM_DSC'] == "NOTIONS & YARN")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains("knit|crochet|needle|stitch kit|ptrn|batting santa sock|sewing|sugar'n cream|c&c|zipper")), ['VML2', 'VRF2']] = 'Needlework & Toy Makibg'
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'CRAFTS/NOTIONS/YARN') & (df_spc['SUBCOM_DSC'] == "NOTIONS & YARN")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains("ribbon")), ['VML1', 'VRF1']] = 'Office Supplies'
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'CRAFTS/NOTIONS/YARN') & (df_spc['SUBCOM_DSC'] == "NOTIONS & YARN")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains("ribbon")), ['VML2', 'VRF2']] = 'Greeting Cards & Gift Packaging'        
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'CRAFTS/NOTIONS/YARN') & (df_spc['SUBCOM_DSC'] == "NOTIONS & YARN")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains("logo patch")), ['VML1', 'VRF1']] = 'Toys / Games / Hobbies'
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'CRAFTS/NOTIONS/YARN') & (df_spc['SUBCOM_DSC'] == "NOTIONS & YARN")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains("logo patch")), ['VML2', 'VRF2']] = 'Games & Puzzles'  
        
     return df_spc     

In [14]:
 def process_gm_5(df_spc): 
     """
       DPT_DSC: SCHOOL/OFFICE
       COM_DSC: CRAFTS/NOTIONS/YARN + NOTIONS & YARN

     """ 

     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'CRAFTS/NOTIONS/YARN') & (df_spc['SUBCOM_DSC'] == "ART & CRAFT SUPPLIES")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('clear stamp|cling stamp|stamp|stencil|gold acryl')), ['VML1', 'VRF1']] = 'Arts & Crafts'
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'CRAFTS/NOTIONS/YARN') & (df_spc['SUBCOM_DSC'] == "ART & CRAFT SUPPLIES")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('clear stamp|cling stamp|stamp|stencil|gold acryl')), ['VML2', 'VRF2']] = 'Drawing Tools'
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'CRAFTS/NOTIONS/YARN') & (df_spc['SUBCOM_DSC'] == "ART & CRAFT SUPPLIES")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('die|love odds & ends|tutti designs|ceramcoat acrylic|airbrush tabletop stand holder|mt gold acryl ancient pink')), ['VML1', 'VRF1']] = 'Arts & Crafts'
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'CRAFTS/NOTIONS/YARN') & (df_spc['SUBCOM_DSC'] == "ART & CRAFT SUPPLIES")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('die|love odds & ends|tutti designs|ceramcoat acrylic|airbrush tabletop stand holder|mt gold acryl ancient pink')), ['VML2', 'VRF2']] = 'Drawing Tools'
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'CRAFTS/NOTIONS/YARN') & (df_spc['SUBCOM_DSC'] == "ART & CRAFT SUPPLIES")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('sticker|doodle pops|big dot of happiness|home sweet haunted|eks jolee')), ['VML1', 'VRF1']] = 'Arts & Crafts'
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'CRAFTS/NOTIONS/YARN') & (df_spc['SUBCOM_DSC'] == "ART & CRAFT SUPPLIES")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('sticker|doodle pops|big dot of happiness|home sweet haunted|eks jolee')), ['VML2', 'VRF2']] = 'Stickers'   
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'CRAFTS/NOTIONS/YARN') & (df_spc['SUBCOM_DSC'] == "ART & CRAFT SUPPLIES")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('vintage sewing machine|craft sticks|glass glitter|feather domestic goose|river rock|embellishment|tassel end|wood letters|home & welcome|garden tags ribbon accents')), ['VML1', 'VRF1']] = 'Arts & Crafts'
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'CRAFTS/NOTIONS/YARN') & (df_spc['SUBCOM_DSC'] == "ART & CRAFT SUPPLIES")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('vintage sewing machine|craft sticks|glass glitter|feather domestic goose|river rock|embellishment|tassel end|wood letters|home & welcome|garden tags ribbon accents')), ['VML2', 'VRF2']] = 'Craft Sticks'  
     
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'CRAFTS/NOTIONS/YARN') & (df_spc['SUBCOM_DSC'] == "ART & CRAFT SUPPLIES")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('knit|quilt|lazy girl designs iron on vinyl|mango madness')), ['VML1', 'VRF1']] = 'Arts & Crafts'
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'CRAFTS/NOTIONS/YARN') & (df_spc['SUBCOM_DSC'] == "ART & CRAFT SUPPLIES")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('knit|quilt|lazy girl designs iron on vinyl|mango madness')), ['VML2', 'VRF2']] = 'Needlework & Toy Makibg'
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'CRAFTS/NOTIONS/YARN') & (df_spc['SUBCOM_DSC'] == "ART & CRAFT SUPPLIES")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('slvr/crys|sft suede lace')), ['VML1', 'VRF1']] = 'Arts & Crafts'
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'CRAFTS/NOTIONS/YARN') & (df_spc['SUBCOM_DSC'] == "ART & CRAFT SUPPLIES")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('slvr/crys|sft suede lace')), ['VML2', 'VRF2']] = 'Jewelry Making'
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'CRAFTS/NOTIONS/YARN') & (df_spc['SUBCOM_DSC'] == "ART & CRAFT SUPPLIES")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('plank|art wood house')), ['VML1', 'VRF1']] = 'Arts & Crafts'
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'CRAFTS/NOTIONS/YARN') & (df_spc['SUBCOM_DSC'] == "ART & CRAFT SUPPLIES")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('plank|art wood house')), ['VML2', 'VRF2']] = 'Paper / Canvas / Wood Boards'    
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] =='CRAFTS/NOTIONS/YARN')  & (df_spc['SUBCOM_DSC'] == "ART & CRAFT SUPPLIES")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('variety pack|value variety|hole punch|signboard|clip stand|bookmark|album')), ['VML1', 'VRF1']] = 'Office Supplies'
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] =='CRAFTS/NOTIONS/YARN')  & (df_spc['SUBCOM_DSC'] == "ART & CRAFT SUPPLIES")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('variety pack|value variety|hole punch|signboard|clip stand|bookmark|album')), ['VML2', 'VRF2']] = 'Stationery' 
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'CRAFTS/NOTIONS/YARN') & (df_spc['SUBCOM_DSC'] == "ART & CRAFT SUPPLIES")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('recipe card')), ['VML1', 'VRF1']] = 'Kitchen & Bathroom'
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'CRAFTS/NOTIONS/YARN') & (df_spc['SUBCOM_DSC'] == "ART & CRAFT SUPPLIES")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('recipe card')), ['VML2', 'VRF2']] = 'Kitchen'
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'CRAFTS/NOTIONS/YARN') & (df_spc['SUBCOM_DSC'] == "ART & CRAFT SUPPLIES")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('wooden bar unit')), ['VML1', 'VRF1']] = 'Kitchen & Bathroom'
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'CRAFTS/NOTIONS/YARN') & (df_spc['SUBCOM_DSC'] == "ART & CRAFT SUPPLIES")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('wooden bar unit')), ['VML2', 'VRF2']] = 'Cabinets'        
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'CRAFTS/NOTIONS/YARN') & (df_spc['SUBCOM_DSC'] == "ART & CRAFT SUPPLIES")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('artificial|pink rose glass cube')), ['VML1', 'VRF1']] = 'Home & Venue Decoration'
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'CRAFTS/NOTIONS/YARN') & (df_spc['SUBCOM_DSC'] == "ART & CRAFT SUPPLIES")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('artificial|pink rose glass cube')), ['VML2', 'VRF2']] = 'Floral & Foliage - Artifical'    
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'CRAFTS/NOTIONS/YARN') & (df_spc['SUBCOM_DSC'] == "ART & CRAFT SUPPLIES")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('plaid wood hanger w/3 crystal knobs')), ['VML1', 'VRF1']] = 'Home & Venue Decoration'
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'CRAFTS/NOTIONS/YARN') & (df_spc['SUBCOM_DSC'] == "ART & CRAFT SUPPLIES")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('plaid wood hanger w/3 crystal knobs')), ['VML2', 'VRF2']] = 'Decorative Accessories'       
       
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'CRAFTS/NOTIONS/YARN') & (df_spc['SUBCOM_DSC'] == "ART & CRAFT SUPPLIES")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('cdstk|doodlebug petite prints buffalo check/wood grain|cardstock|card stock|photoplay collection pack|colorplay collection pack|echo park|super satchel|scenescapes sea breeze|water droplet embellishments|med root beer|no place like home')), ['VML1', 'VRF1']] = 'Arts & Crafts'
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'CRAFTS/NOTIONS/YARN') & (df_spc['SUBCOM_DSC'] == "ART & CRAFT SUPPLIES")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('cdstk|doodlebug petite prints buffalo check/wood grain|cardstock|card stock|photoplay collection pack|colorplay collection pack|echo park|super satchel|scenescapes sea breeze|water droplet embellishments|med root beer|no place like home')), ['VML2', 'VRF2']] = 'Paper & Card Making'
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'CRAFTS/NOTIONS/YARN') & (df_spc['SUBCOM_DSC'] == "ART & CRAFT SUPPLIES")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('canvas')), ['VML1', 'VRF1']] = 'Arts & Crafts'
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'CRAFTS/NOTIONS/YARN') & (df_spc['SUBCOM_DSC'] == "ART & CRAFT SUPPLIES")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('canvas')), ['VML2', 'VRF2']] = 'Paper / Canvas / Wood Boards'  
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'CRAFTS/NOTIONS/YARN') & (df_spc['SUBCOM_DSC'] == "ART & CRAFT SUPPLIES")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('alcohol ink')), ['VML1', 'VRF1']] = 'Office Supplies'
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'CRAFTS/NOTIONS/YARN') & (df_spc['SUBCOM_DSC'] == "ART & CRAFT SUPPLIES")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('alcohol ink')), ['VML2', 'VRF2']] = 'Stationery'    
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'CRAFTS/NOTIONS/YARN') & (df_spc['SUBCOM_DSC'] == "ART & CRAFT SUPPLIES")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('wooden links|premium rose head|beaded cyrstal ornament kit')), ['VML1', 'VRF1']] = 'Home & Venue Decoration'
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'CRAFTS/NOTIONS/YARN') & (df_spc['SUBCOM_DSC'] == "ART & CRAFT SUPPLIES")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('wooden links|premium rose head|beaded cyrstal ornament kit')), ['VML2', 'VRF2']] = 'Decorative Accessories'  
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'CRAFTS/NOTIONS/YARN') & (df_spc['SUBCOM_DSC'] == "ART & CRAFT SUPPLIES")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('keyboard stand')), ['VML1', 'VRF1']] = 'Books & Videos'
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'CRAFTS/NOTIONS/YARN') & (df_spc['SUBCOM_DSC'] == "ART & CRAFT SUPPLIES")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('keyboard stand')), ['VML2', 'VRF2']] = 'Music'         
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'CRAFTS/NOTIONS/YARN') & (df_spc['SUBCOM_DSC'] == "ART & CRAFT SUPPLIES")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('bird house kit')), ['VML1', 'VRF1']] = 'Gardening & Outdoors'
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'CRAFTS/NOTIONS/YARN') & (df_spc['SUBCOM_DSC'] == "ART & CRAFT SUPPLIES")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('bird house kit')), ['VML2', 'VRF2']] = 'Outdoor Living'    
      
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'CRAFTS/NOTIONS/YARN') & (df_spc['SUBCOM_DSC'] == "ART & CRAFT SUPPLIES")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains("template|rub ' n buff|story card|wood crate set|diy")), ['VML1', 'VRF1']] = 'Arts & Crafts'
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'CRAFTS/NOTIONS/YARN') & (df_spc['SUBCOM_DSC'] == "ART & CRAFT SUPPLIES")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains("template|rub ' n buff|story card|wood crate set|diy")), ['VML2', 'VRF2']] = 'Misc'        
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'CRAFTS/NOTIONS/YARN') & (df_spc['SUBCOM_DSC'] == "ART & CRAFT SUPPLIES")  & (df_spc['VND_ECOM_DSC'].str.contains('SIMPLY WILD WEEKENDER')), ['VML2', 'VRF2']] = 'Misc' 
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'CRAFTS/NOTIONS/YARN') & (df_spc['SUBCOM_DSC'] == "ART & CRAFT SUPPLIES")  & (df_spc['VND_ECOM_DSC'].str.contains('SIMPLY WILD WEEKENDER')), ['VML1', 'VRF1']] = 'Misc'
     return df_spc  

In [15]:
 def process_gm_6(df_spc): 
     """
       DPT_DSC: SCHOOL/OFFICE
       COM_DSC: HOME AND OFFICE

     """ 
      
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'].str.contains('HOME AND OFFICE')) & (df_spc['SUBCOM_DSC'].str.contains("MESSAGE CENTER"))  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('dry erase|hole punch|board')), ['VML1', 'VRF1']] = 'Office Supplies'
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'].str.contains('HOME AND OFFICE')) & (df_spc['SUBCOM_DSC'].str.contains("MESSAGE CENTER"))  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('dry erase|hole punch|board')), ['VML2', 'VRF2']] = 'Stationery'
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'].str.contains('HOME AND OFFICE')) & (df_spc['SUBCOM_DSC'].str.contains("MESSAGE CENTER"))  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('dry erase|hole punch|board')), ['VML1', 'VRF1']] = 'Office Supplies'
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'].str.contains('HOME AND OFFICE')) & (df_spc['SUBCOM_DSC'].str.contains("MESSAGE CENTER"))  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('dry erase|hole punch|board')), ['VML2', 'VRF2']] = 'Stationery'  
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'].str.contains('HOME AND OFFICE')) & (df_spc['SUBCOM_DSC'].str.contains("MESSAGE CENTER"))  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('lamp')), ['VML1', 'VRF1']] = 'Office Supplies'
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'].str.contains('HOME AND OFFICE')) & (df_spc['SUBCOM_DSC'].str.contains("MESSAGE CENTER"))  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('lamp')), ['VML2', 'VRF2']] = 'Office Machinery'  
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'].str.contains('HOME AND OFFICE')) & (df_spc['SUBCOM_DSC'].str.contains("MESSAGE CENTER"))  & (df_spc['VND_ECOM_DSC'].str.contains('MEGA 22X18 DE CAL CORX MAG')), ['VML1', 'VRF1']] = 'Misc'
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'].str.contains('HOME AND OFFICE')) & (df_spc['SUBCOM_DSC'].str.contains("MESSAGE CENTER"))  & (df_spc['VND_ECOM_DSC'].str.contains('MEGA 22X18 DE CAL CORX MAG')), ['VML2', 'VRF2']] = 'Misc'    
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'HOME AND OFFICE') & (df_spc['SUBCOM_DSC'] == "OFFICE ACCESSORIES")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains("pwr calc")), ['VML1', 'VRF1']] = 'Office Supplies'
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'HOME AND OFFICE') & (df_spc['SUBCOM_DSC'] == "OFFICE ACCESSORIES")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains("pwr calc")), ['VML2', 'VRF2']] = 'Office Machinery'
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'HOME AND OFFICE') & (df_spc['SUBCOM_DSC'] == "OFFICE ACCESSORIES")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains("rose globe")), ['VML1', 'VRF1']] = 'Home & Venue Decoration'
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'HOME AND OFFICE') & (df_spc['SUBCOM_DSC'] == "OFFICE ACCESSORIES")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains("rose globe")), ['VML2', 'VRF2']] = 'Decorative Accessories'    
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'HOME AND OFFICE')  & (df_spc['SUBCOM_DSC'] == "OFFICE PAPER")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('sticky note holder')), ['VML1', 'VRF1']] = 'Office Supplies'
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'HOME AND OFFICE')  & (df_spc['SUBCOM_DSC'] == "OFFICE PAPER")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('sticky note holder')), ['VML2', 'VRF2']] = 'Stationery' 
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'HOME AND OFFICE')  & (df_spc['SUBCOM_DSC'] == "OFFICE PAPER")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('cardstock')), ['VML1', 'VRF1']] = 'Arts & Crafts'
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'HOME AND OFFICE')  & (df_spc['SUBCOM_DSC'] == "OFFICE PAPER")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('cardstock')), ['VML2', 'VRF2']] = 'Paper & Card Making'
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'HOME AND OFFICE') & (df_spc['SUBCOM_DSC'] == "OFFICE PAPER")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('sticky note holder')), ['VML1', 'VRF1']] = 'Office Supplies'
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'HOME AND OFFICE') & (df_spc['SUBCOM_DSC'] == "OFFICE PAPER")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('sticky note holder')), ['VML2', 'VRF2']] = 'Stationery' 
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'HOME AND OFFICE') & (df_spc['SUBCOM_DSC'] == "OFFICE PAPER")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('cardstock')), ['VML1', 'VRF1']] = 'Arts & Crafts'
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'HOME AND OFFICE') & (df_spc['SUBCOM_DSC'] == "OFFICE PAPER")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('cardstock')), ['VML2', 'VRF2']] = 'Paper & Card Making'        
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'HOME AND OFFICE') & (df_spc['SUBCOM_DSC'] == "TAPE & MAILING PRODUCTS")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('shipping box')), ['VML1', 'VRF1']] = 'Office Supplies'
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'HOME AND OFFICE') & (df_spc['SUBCOM_DSC'] == "TAPE & MAILING PRODUCTS")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('shipping box')), ['VML2', 'VRF2']] = 'Shipping and Packaging' 
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'HOME AND OFFICE') & (df_spc['SUBCOM_DSC'] == "LABELS/POST-IT/INDEX CARD")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('signmission|post-it flags|label')), ['VML1', 'VRF1']] = 'Office Supplies'
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'HOME AND OFFICE') & (df_spc['SUBCOM_DSC'] == "LABELS/POST-IT/INDEX CARD")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('signmission|post-it flags|label')), ['VML2', 'VRF2']] = 'Sationary'
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'HOME AND OFFICE') & (df_spc['SUBCOM_DSC'] == "LABELS/POST-IT/INDEX CARD")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('slim pack')), ['VML1', 'VRF1']] = 'Office Supplies'
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'HOME AND OFFICE') & (df_spc['SUBCOM_DSC'] == "LABELS/POST-IT/INDEX CARD")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('slim pack')), ['VML2', 'VRF2']] = 'Organizational Planning Stationery'        
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'HOME AND OFFICE') & (df_spc['SUBCOM_DSC'] == "LABELS/POST-IT/INDEX CARD")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('banner stand')), ['VML1', 'VRF1']] = 'Office Supplies'
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'HOME AND OFFICE') & (df_spc['SUBCOM_DSC'] == "LABELS/POST-IT/INDEX CARD")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('banner stand')), ['VML2', 'VRF2']] = 'Office Machinery'        
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'HOME AND OFFICE') & (df_spc['SUBCOM_DSC'] == "PLANNERS & JOURNALS")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('cor display 0001 single tier')), ['VML1', 'VRF1']] = 'Misc'
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'HOME AND OFFICE') & (df_spc['SUBCOM_DSC'] == "PLANNERS & JOURNALS")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('cor display 0001 single tier')), ['VML2', 'VRF2']] = 'Misc'        
     
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'SCHOOL') & (df_spc['SUBCOM_DSC'] == "SCHOOL SUPPLIES")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('chalkboard wall hanging')), ['VML1', 'VRF1']] = 'Home & Venue Decoration'
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'SCHOOL') & (df_spc['SUBCOM_DSC'] == "SCHOOL SUPPLIES")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('chalkboard wall hanging')), ['VML2', 'VRF2']] = 'Decorative Accessories' 
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'SCHOOL') & (df_spc['SUBCOM_DSC'] == "SCHOOL SUPPLIES")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('chartlet|accents|variety pack|pencil')), ['VML1', 'VRF1']] = 'Office Supplies'
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'SCHOOL') & (df_spc['SUBCOM_DSC'] == "SCHOOL SUPPLIES")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('chartlet|accents|variety pack|pencil')), ['VML2', 'VRF2']] = 'Sationary'
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'SCHOOL') & (df_spc['SUBCOM_DSC'] == "SCHOOL SUPPLIES")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('globe|bulletin board')), ['VML1', 'VRF1']] = 'Office Supplies'
     df_spc.loc[(df_spc["DPT_DSC"] == 'SCHOOL/OFFICE') & (df_spc['COM_DSC'] == 'SCHOOL') & (df_spc['SUBCOM_DSC'] == "SCHOOL SUPPLIES")  & (df_spc['VND_ECOM_DSC'].str.lower().str.contains('globe|bulletin board')), ['VML2', 'VRF2']] = 'Office Machinery'        
        
     return df_spc

In [16]:
import numpy as np
import pandas as pd
 

pd.options.display.float_format = '{:,.4f}'.format
pd.set_option('display.max_columns', None) 
pd.set_option("display.max_rows", None)
path = 'C:\\users\\iny2819\\kroger\\Data\\'

In [17]:
df_org = data_preparation_1()
df, l_new_var = data_preparation_2(df_org)
df_bkup = df.copy()
df_org_bkup = df_org.copy()


In [18]:
df_beer = process_beer(df)
df_beer.MANU_VAL_REQ.value_counts()

NO     20200
YES     3173
Name: MANU_VAL_REQ, dtype: int64

In [19]:
df_spirits = process_spirits(df)
df_spirits.MANU_VAL_REQ.value_counts()

NO     19419
YES     2984
Name: MANU_VAL_REQ, dtype: int64

In [20]:
df_wine = process_wine(df)
df_wine.MANU_VAL_REQ.value_counts() 

NO     32244
YES     1501
Name: MANU_VAL_REQ, dtype: int64

In [21]:
df_hbc = process_hbc(df)
df_hbc.MANU_VAL_REQ.value_counts() 

YES    132
Name: MANU_VAL_REQ, dtype: int64

In [22]:
df_gm = process_gm(df)
df_gm.MANU_VAL_REQ.value_counts() 

YES    2376
Name: MANU_VAL_REQ, dtype: int64

In [23]:
df_gm.shape

(2376, 21)

In [24]:
df_gm ['VML1'].value_counts()

Kitchen & Bathroom                          843
Home & Venue Decoration                     582
Arts & Crafts                               514
Office Supplies                             246
???                                          47
Books & Videos                               40
Gardening & Outdoors                         32
Electronics                                  31
Automotive                                    8
Food / Beverages                              7
Misc                                          5
Apparel                                       4
Toys / Games / Hobbies                        3
Livestock & Pet Supplies                      3
Hospitality Supplies                          3
Beer / Wine / Spirits                         3
Elec                                          2
Electrical                                    1
Sports & Outdoor Recreation Equipment         1
Heating / Ventilation / Air Conditioning      1
Name: VML1, dtype: int64

In [25]:
df_gm ['VML2'].value_counts() 

Kitchen                               825
Decorative Accessories                375
Drawing Tools                         177
Paper & Card Making                   151
Greeting Cards & Gift Packaging       130
Special Occasion & Party Supplies     108
Needlework & Toy Makibg               107
Stationery                             85
Holiday Decorations                    64
Stickers                               50
???                                    47
Film & Television                      39
Gardening                              21
Candles & Accessories                  21
Computers                              20
Cabinets                               17
Sationary                              15
Craft Sticks                           13
Misc                                   13
Office Machinery                       12
Floral & Foliage - Artifical           11
Audio & Video                          11
Outdoor Living                         10
Grocery                           

In [33]:
merged_file = merge_file(df_org)

PermissionError: [Errno 13] Permission denied: 'C:\\users\\iny2819\\kroger\\Data\\Beer_Wine_Spirits_ML_RF_val.xlsx'

In [27]:
merged_file ['VML1'].value_counts() 

Beer / Wine / Spirits                       79524
Kitchen & Bathroom                            843
Home & Venue Decoration                       582
Arts & Crafts                                 514
Office Supplies                               261
Health & Beauty                                88
???                                            47
Books & Videos                                 40
Gardening & Outdoors                           32
Electronics                                    31
Food / Beverages                               20
Tobacco Products                               16
Automotive                                      8
Misc                                            5
Apparel                                         4
Toys / Games / Hobbies                          3
Livestock & Pet Supplies                        3
Hospitality Supplies                            3
Elec                                            2
Electrical                                      1


In [28]:
merged_file ['VML2'].value_counts()

Wine                                  33582
Spirits                               20189
Beer                                  19882
Flavored Alcoholic Beverages           2572
Flavored Alcoholic Beveragess          2214
Ciders & Perrys                        1082
Kitchen                                 825
Decorative Accessories                  375
Drawing Tools                           177
Paper & Card Making                     151
Greeting Cards & Gift Packaging         145
Special Occasion & Party Supplies       108
Needlework & Toy Makibg                 107
Stationery                               85
Cosmetics & Fragrances                   75
Holiday Decorations                      64
Stickers                                 50
???                                      47
Film & Television                        39
Gardening                                21
Candles & Accessories                    21
Computers                                20
Cabinets                        

In [29]:
merged_file.shape

(82580, 34)

In [30]:
Stop

NameError: name 'Stop' is not defined

In [ ]:
l_val_df= [dfx, df_beer, df_spirits, df_wine, df_hbc, df_gm]
df_val = pd.concat(l_val_df, ignore_index=True, sort=False)

In [ ]:
    keep_col = ['row_num','VML1', 'VML2', 'VRF1', 'VRF2', 'VML1_MTH_IND', 'VML2_MTH_IND', 'VRF1_MTH_IND', 'VRF2_MTH_IND',
               'MANU_VAL_REQ', 'MANU_VAL_DONE']
      
    df_val = df_val[keep_col]

In [ ]:
df_merged = pd.merge(df_org,
                    df_val,
                    on='row_num', how='left')

In [ ]:
path = 'C:\\users\\iny2819\\kroger\\Data\\'
df_merged.to_csv(path + "Beer_Wine_Spirits_ML_RF_val.csv",index=None)

In [ ]:
df_beer['VML1_MTH_IND'].value_counts()


In [ ]:
df_beer['VML2_MTH_IND'].value_counts()

In [ ]:
df_beer['VRF1_MTH_IND'].value_counts()

In [ ]:
df_beer['VRF2_MTH_IND'].value_counts()

In [ ]:
df_beer.loc[(df_beer['VML1_IND'] == False) |(df_beer['VML2_IND'] == False) |(df_beer['VRF1_IND'] == False) |(df_beer['VRF2_IND'] == False)  , 'MANU_VAL_IND'] = 'True'

In [ ]:
df_gm['VML1'].value_counts()

In [ ]:
stop

In [ ]:
df[df["PMY_DPT_DSC"] == 'MEAT']["REC_DPT_DSC"].value_counts()

In [ ]:
df[df["PMY_DPT_DSC"] == 'DELI/BAKE']["REC_DPT_DSC"].value_counts()

In [ ]:
df[df["PMY_DPT_DSC"] == 'DRUG/GM']["REC_DPT_DSC"].value_counts()

In [ ]:
df[(df["PMY_DPT_DSC"] == 'DRUG/GM') & (df["REC_DPT_DSC"] == 'GM')]["DPT_DSC"].value_counts()

In [ ]:
df[df["PMY_DPT_DSC"] == 'GROCERY']["REC_DPT_DSC"].value_counts()


In [ ]:
df[(df["PMY_DPT_DSC"] == 'GROCERY') & (df["REC_DPT_DSC"] == 'LIQUOR')]["DPT_DSC"].value_counts()

## wip

In [ ]:
type(df[df["PMY_DPT_DSC"] == 'GROCERY'])

In [ ]:
df.columns

In [ ]:
gtin = df_val['row_num'].value_counts(dropna = False)
len(gtin) 

In [ ]:
gtin = gtin[gtin != 1]
len(gtin) #  duplicate gtin = 3326

In [ ]:
gtin = df_org['row_num'].value_counts(dropna = False)
len(gtin) 

In [ ]:
gtin = gtin[gtin != 1]
len(gtin) #  duplicate gtin = 3326

In [ ]:
df_val.info()

In [ ]:
print(l_new_var)

In [ ]:
df_merged.head(2)
F

In [ ]:
    df_beer.loc[(df_beer['VML1_MTH_IND'] == False) |(df_beer['VML2_MTH_IND'] == False) |(df_beer['VRF1_MTH_IND'] == False) |(df_beer['VRF2_MTH_IND'] == False)  , 'MANU_VAL_REQ'] = 'YES'

In [ ]:
#df_spc1 = df_gm.loc[df_gm['VML2'] == '???']
df_spc1 = df_gm.loc[(df_gm['VML1'] != '???') & (df_gm['VML2'] == '???')]
path = 'C:\\users\\iny2819\\kroger\\Data\\'
df_spc1.to_excel(path + "wip_ML_RF_val.xlsx",index=None)

In [ ]:
df_spc1.shape

In [ ]:
 df_gm.loc[(df_gm['VML1'] == '???') &  (df_gm["DPT_DSC"] == "HOUSEWARES")]["COM_DSC"].value_counts(dropna = False)

In [ ]:
df.shape

In [ ]:
df_val.shape

In [ ]:
dfx = df.copy()

In [ ]:

dfx["ind"] = (~df.row_num.isin(df_val.row_num))
dfx = dfx.loc[dfx["ind"] == True]
dfx = dfx.drop(['ind'], axis = 1)


In [ ]:
dfx.head()

In [ ]:
#l_val_df= [dfx, df_beer, df_spirits, df_wine, df_hbc, df_gm]
l_val_df= [dfx, df_hbc]
df_val = pd.concat(l_val_df, ignore_index=True, sort=False)

In [ ]:
dfx.columns


In [ ]:
df_gm.columns


In [ ]:
df_spc1 = df_gm.loc[(df_gm['VML1'] == '???' ) & (df_gm['VML2'] != '???' )]
df_spc1.to_excel(path + "wip_ML_RF_val.xlsx",index=None)

In [ ]:
df_hbc.columns

In [ ]:

path = 'C:\\users\\iny2819\\kroger\\Data\\'
df_gm.to_excel(path + "wip_ML_RF_val.xlsx",index=None)

In [31]:
df['VML1'].value_counts() 

???    82580
Name: VML1, dtype: int64

In [32]:
df['VML2'].value_counts() 

???    82580
Name: VML2, dtype: int64